In [2]:
# Imports para os exercícios

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableBranch, RunnableLambda, RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from pydantic import BaseModel, Field

# Carregando arquivos de configuração
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


# Desafios de Langchain

## Desafio 1
Crie uma chain que recebe o nome de um país e retorna 3 curiosidades sobre ele

In [ ]:
country_template = ChatPromptTemplate.from_template("Cite 3 curiosidades sobre {country}")
country_model = ChatOpenAI(model='gpt-4o-mini')
country_parser = StrOutputParser()

country_chain = country_template | country_model | country_parser
country_chain.invoke({"country":"Austrália"})

## Desafio 2

Crie duas chains: a primeira gera uma receita a partir de uma lista de ingredientes, a segunda avalia a receita e sugere melhorias

In [ ]:
recipe_template = ChatPromptTemplate.from_template("Me sugira uma receita com os seguintes ingredientes: {ingredientes}")
model = ChatOpenAI(model='gpt-4o-mini')
parser = StrOutputParser()

recipe_chain = recipe_template | model | parser

ingredientes = [
    "Arroz",
    "Batata",
    "Carne Moída"
]

improve_recipe_template = ChatPromptTemplate.from_template("Sugira melhorias para a seguinte receita: {receita}")

improve_recipe_chain = improve_recipe_template | model | parser

generate_recipe_chain = recipe_chain | improve_recipe_chain
generate_recipe_chain.invoke({"ingredientes":ingredientes})


## Desafio 3 

Crie uma chain que recebe a descrição de um problema, classifica a categoria (bug, feature request, dúvida) usando structured output com Pydantic, e depois gera uma resposta adequada ao tipo

In [ ]:
class IdentifyRequest(BaseModel):
    """"Recebe uma string de usuário e retorna a categoria"""""
    category: str = Field(description="Define a categoria da mensagem do usuário")

model = ChatOpenAI(model='gpt-4o-mini')
parser = StrOutputParser()

received_request_template = ChatPromptTemplate.from_template("Você receberá uma mensagem de um usuário e deve classificá-la ou como bug ou feature request ou dúvida: {mensagem}")

chat_identify_request = model.with_structured_output(IdentifyRequest)

mensagem = "O botão de logar deveria ser azul" 

identify_request_chain = received_request_template | chat_identify_request 

process_request_template = ChatPromptTemplate.from_template("Você recebe a seguinte mensagem do usuário: '{mensagem}'. Essa mensagem foi categorizada como {category}. Forneça uma resposta adequada ao usuário")

process_request_chain = process_request_template | model | parser

runnable_identify_request = RunnableParallel(category=identify_request_chain | (lambda x: x.category), mensagem=lambda x: x['mensagem'])

request_chain = runnable_identify_request | process_request_chain

request_chain.invoke({"mensagem":mensagem})


## Desafio 4


Expanda o exercício anterior: dependendo da categoria classificada, a chain deve seguir por caminhos diferentes. Bug vai pra um prompt que pede passos de reprodução, feature request vai pra um prompt que pede justificativa de negócio, dúvida vai pra um prompt que responde diretamente. 

In [ ]:
# definir os prompts e chains de acordo com a categorização

default_chain = RunnableLambda(lambda _: "Não entendi.")

bug_prompt_template = ChatPromptTemplate.from_template("Leia o bug report e gere uma resposta ao usuário pedindo para explicar como reproduzir o problema: {mensagem}")
bug_chain = bug_prompt_template | model | parser

feature_request_template = ChatPromptTemplate.from_template("Leia a feature request do usuário e gere uma resposta pedindo uma justificativa ao usuário para a solicitação: {mensagem}")
feature_request_chain = feature_request_template | model | parser

question_template = ChatPromptTemplate.from_template("Leia a dúvida e procure responder a dúvida do usuário: {mensagem}. Caso não saiba, diga que não sabe mesmo")
question_chain = question_template | model | parser

branch_chain = RunnableBranch(
    (lambda x: x.get("category") == "bug", bug_chain),
    (lambda x: x.get("category") == "feature request", feature_request_chain),
    (lambda x: x.get("category") == "duvida", question_chain),
    default_chain
)

new_chain = runnable_identify_request | branch_chain 
new_chain.invoke({"mensagem":mensagem})

## Desafio 5

Crie um chatbot que mantém contexto de conversa. O usuário pode pedir recomendações de filmes, e o bot deve lembrar o que já recomendou e quais preferências o usuário mencionou

In [ ]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Você é um cinéfilo que ajuda usuários indicando filmes para os usuários, baseado no que eles pediram"),
    MessagesPlaceholder(variable_name="historico"),
    ("human","{duvida}")
])

model = ChatOpenAI(model='gpt-4o-mini')

main_chain = chat_prompt_template | model

memory = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in memory:
        memory[session_id] = InMemoryChatMessageHistory()
    return memory[session_id]

chain_with_history = RunnableWithMessageHistory(
    runnable=main_chain,
    get_session_history=get_session_history,
    input_messages_key="duvida",
    history_messages_key="historico"
)

complete_chain = chain_with_history | parser
complete_chain.invoke({
    "duvida":"Me cite 5 filmes de terror"
},{"configurable":{"session_id":"user123"}})



Testando se ele considera o histórico da conversa

In [ ]:
complete_chain.invoke({
    "duvida":"Qual destes é o mais assustador?"
},{"configurable":{"session_id":"user123"}})

## Desafio 6

Crie uma chain que carrega um documento PDF, divide em chunks, armazena num vector store (pode usar FAISS ou Chroma), e responde perguntas sobre o conteúdo do documento. Esse exercício junta embeddings, retriever e chain de QA, e é o padrão mais demandado no mercado pra quem trabalha com LLM em produto.

In [4]:
# Importando o PDF
faq_loader = PyPDFLoader("../arquivos/faq.pdf")
faq = faq_loader.load()

In [ ]:
#dividindo em chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False
)

chunk_faq = text_splitter.create_documents(faq)

In [ ]:
#criando os embeddings

vector_faq = FAISS.from_documents(chunk_faq, OpenAIEmbeddings())

In [ ]:
#criando o retriever

faq_retriever = vector_faq.as_retriever()

In [ ]:
#função para juntar os documentos de chunks 

def format_docs(docs):
    return "\n".join(doc.page_content for doc in docs)

In [ ]:
faq_prompt = ChatPromptTemplate.from_template("Com base no seguinte contexto: '{contexto}' Responda a pergunta do usuário: {pergunta}")

runnable_find_document = RunnableParallel(
    contexto = (lambda x: x['pergunta']) | faq_retriever | format_docs, pergunta = lambda x: x["pergunta"]
)

faq_chain = runnable_find_document | faq_prompt | model | parser
faq_chain.invoke({"pergunta":"O sistema funciona offline?"})